In [3]:
import time
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_classic.storage import LocalFileStore
from langchain_huggingface import HuggingFaceEmbeddings

# Create your underlying embeddings model
underlying_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Store persists embeddings to the local filesystem
# This isn't for production use, but is useful for local
store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace="sentence-transformers/all-MiniLM-L6-v2"
)

# Example: caching a query embedding
tic = time.time()
print(cached_embedder.embed_query("Hello, world!"))
print(f"First call took: {time.time() - tic:.2f} seconds")

# Subsequent calls use the cache
tic = time.time()
print(cached_embedder.embed_query("Hello, world!"))
print(f"Second call took: {time.time() - tic:.2f} seconds")

[-0.03817712888121605, 0.032911114394664764, -0.005459363106638193, 0.014369938522577286, -0.04029100760817528, -0.11653248220682144, 0.03168764337897301, 0.0019117374904453754, -0.04262235388159752, 0.02916811965405941, 0.042426664382219315, 0.03204170987010002, 0.029844731092453003, 0.010980322025716305, -0.0539395846426487, -0.05027728155255318, -0.02350781112909317, 0.010779324918985367, -0.13770791888237, 0.004115021787583828, 0.02933308109641075, 0.06684113293886185, -0.01538940705358982, 0.04843765124678612, -0.08814972639083862, -0.01272681076079607, 0.041409045457839966, 0.04083149507641792, -0.0501558780670166, -0.05812504142522812, 0.04880151525139809, 0.06889012455940247, 0.05874689668416977, 0.008730987086892128, -0.015918249264359474, 0.08514200150966644, -0.07814744859933853, -0.07751677185297012, 0.020723793655633926, 0.016194242984056473, 0.03251060098409653, -0.05348888412117958, -0.062228769063949585, -0.02431463822722435, 0.007412788458168507, 0.02397775836288929, 0

/home/ubuntu/dev/cora_customer_agent/.venv/lib/python3.13/site-packages/langchain_classic/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()
